**Reference**

This notebook was inspired by [@yuanzhezhou](https://www.kaggle.com/yuanzhezhou)'s [pairwise model](https://www.kaggle.com/code/yuanzhezhou/ai4code-pairwise-bertsmall-inference) and improved upon it.

**about this notebook**

On both Kaggle and Colab, training and inference can be run on this single notebook!

1. Training
    
    Set `CFG.train = True` and run.

2. Inference

    Set `CFG.train = False` and run.

In [ ]:
!nvidia-smi

Mon Aug  8 08:30:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Libraries

In [ ]:
# ====================================================
# import libraries1
# ====================================================

import warnings
warnings.filterwarnings('ignore')

import re
import json
import sys
import os
import numpy as np
import pandas as pd
import gc
import time
import torch
import random
import sys
from tqdm import tqdm
import json
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm
import os
from pathlib import Path
import itertools
import torch.nn.functional as F
import torch.nn as nn
import torch
import collections
import joblib
from gensim.models import word2vec
import datetime
from datetime import timedelta
from sklearn.model_selection import GroupShuffleSplit
import hashlib
from torch.utils.data import DataLoader, Dataset
import torch
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from requests import get
from collections import Counter
from PIL import Image
from glob import glob
import json
from pathlib import Path
from bisect import bisect
import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm
import math
import os
from pathlib import Path
import pickle
from contextlib import contextmanager
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau, _LRScheduler
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV
import lightgbm as lgb
import typing as tp
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import multiprocessing
import difflib
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter

tqdm.pandas()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

Using device: cuda


# Config

In [ ]:
# ==============================================
#  Config
# ==============================================
class CFG:
    colab = "google.colab" in sys.modules
    exp = "051"
    train = False
    debug = False
    api_path = '/content/drive/My Drive/kaggle.json'
    seed = 111
    used_fold = [0]
    fold = 15
    data_split = 50
    # ====================================================
    # Model
    # ====================================================
    model = "prajjwal1/bert-small" # ["microsoft/codebert-base-mlm", "distilbert-base-uncased", "huggingface/CodeBERTa-small-v1", "codistai/codeBERT-small-v2"]
    batch_size = 64
    max_length = 320
    epochs = 5
    num_workers = 8
    lr = 3e-5
    scheduler='linear'

if not CFG.colab:
    CFG.model = "../input/bert-small"

In [ ]:
if CFG.colab:
    print("==============================================")
    print("This environment is Google Colab")
    print("==============================================")

    # Google Drive
    from google.colab import drive, files
    drive.mount('/content/drive')
    %cd "drive/My Drive/AI4Code/"

    # Kaggle API
    f = open(CFG.api_path, 'r')
    json_data = json.load(f) 
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]

    # Directory Setting
    if not os.path.exists(f"output/exp{CFG.exp}/"):
        os.makedirs(f"output/exp{CFG.exp}/")
    
    DATA_DIR = Path("input")
    OUTPUT_DIR = f"output/exp{CFG.exp}/"
    MODEL_DIR = OUTPUT_DIR
    
    # Data Loading
    if not os.path.isfile(os.path.join(DATA_DIR, "AI4Code.zip")):
        !kaggle competitions download -c AI4Code -p $DATA_DIR

    # Libraries
    !pip install -q transformers


else:
    print("==============================================")
    print(" This environment is Kaggle Notebook")
    print("==============================================")

    # Directory Setting
    DATA_DIR = Path("../input/AI4Code/")
    OUTPUT_DIR = "./"
    MODEL_DIR = f"../input/ai4code-dataset-exp{CFG.exp}/"

    # Libraries


# ====================================================
# import libraries2
# ====================================================
from transformers import AutoModel, AutoConfig, AutoTokenizer, AdamW, get_linear_schedule_with_warmup

 This environment is Kaggle Notebook


# Helper Functions

In [ ]:
@contextmanager
def timer(name: str):
    t0 = time.time()
    print(f"[{name}] start")
    yield
    msg = f"[{name}] done in {time.time() - t0:.0f} s"
    print(msg)

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

seed_everything(CFG.seed)

In [ ]:
def init_logger(log_file=OUTPUT_DIR+'train.log'):
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    logger.hasHandlers()
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()

In [ ]:
def cos_sim(v1, v2):
    return np.sum(v1*v2, 1) / (np.linalg.norm(v1, axis=1) * np.linalg.norm(v2, axis=1))

In [ ]:
def gc_clear():
    for i in range(5):
        gc.collect()

In [ ]:
def flatten(x):
    return [e for i in x for e in i]

In [ ]:
def read_notebook(path):
    return (
        pd.read_json(
            path,
            dtype={'cell_type': 'category', 'source': 'str'})
            .assign(id=path.stem)
            .rename_axis('cell_id')
    )

# Evaluation Metric

In [ ]:
# ==============================================
#  Kendall tau correlation
# ==============================================

def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):
        j = bisect(sorted_so_far, u)
        inversions += i - j
        sorted_so_far.insert(j, u)
    return inversions


def kendall_tau(ground_truth, predictions):
    total_inversions = 0
    total_2max = 0
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max

# Data Loading

In [ ]:
with timer("Data Loading"):

    orders_df = pd.read_csv(
                            DATA_DIR / 'train_orders.csv',
                            index_col='id',
                            squeeze=True,
                        ).str.split()

    ancestors_df = pd.read_csv(DATA_DIR / "train_ancestors.csv", index_col='id')
    sub_df = pd.read_csv(DATA_DIR / "sample_submission.csv")

[Data Loading] start
[Data Loading] done in 3 s


In [ ]:
if CFG.train:
    # use downloaded dataframe to save time
    original_df = pd.read_pickle(DATA_DIR / "df.pkl")
else:
    paths_train = list((DATA_DIR / "test").glob("*.json"))
    notebooks_train = [
        read_notebook(path) for path in tqdm(paths_train)
    ]

    original_df = (
        pd.concat(notebooks_train)
            .set_index('id', append=True)
            .swaplevel()
            .sort_index(level='id', sort_remaining=False)
    ).reset_index()

display(original_df)

100%|██████████| 4/4 [00:00<00:00, 86.87it/s]


,id,cell_id,cell_type,source
0,0009d135ece78d,ddfd239c,code,import numpy as np # linear algebra\nimport pa...
1,0009d135ece78d,c6cd22db,code,df = pd.read_csv('/kaggle/input/breast-cancer-...
2,0009d135ece78d,1372ae9b,code,"numerical_data = df.loc[:, ~df.columns.isin(['..."
3,0009d135ece78d,90ed07ab,code,"def comparison_plot_maker(data_1, data_2, name..."
4,0009d135ece78d,7f388a41,code,# Ploting data with different columns\n#######...
5,0009d135ece78d,2843a25a,code,# Scaling Data\nscaler = StandardScaler()\nsca...
6,0009d135ece78d,06dbf8cf,code,# 3. Implementing PCA on X (green for benign; ...
7,0009d135ece78d,f9893819,markdown,# Scaling Data ⚖\nLet's scale the data so PCA ...
8,0009d135ece78d,ba55e576,markdown,## Testing Plots >w>\nLet's these mystery soli...
9,0009d135ece78d,39e937ec,markdown,"## Plotting PCA 📊\nThus, the sun boils down to..."


In [ ]:
if CFG.train:
    orders_df_ = orders_df.to_frame().join(
        original_df.reset_index('cell_id').groupby('id')['cell_id'].apply(list),
        how='right',
    )
    display(orders_df_)

In [ ]:
if CFG.train:
    ranks = {}
    for id_, cell_order, cell_id in orders_df_.itertuples():
        ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id)}
    df_ranks = (
        pd.DataFrame
            .from_dict(ranks, orient='index')
            .rename_axis('id')
            .apply(pd.Series.explode)
            .set_index('cell_id', append=True)
    )

    display(df_ranks)

In [ ]:
if CFG.train:
    original_df = original_df.reset_index().merge(df_ranks, on=["id", "cell_id"]).merge(ancestors_df, on=["id"])
    original_df = original_df.sort_values(["id", "rank"]).reset_index(drop=True)
else:
    original_df["rank"] = original_df.groupby(["id", "cell_type"]).cumcount()
    original_df["ancestor_id"] = 0
display(original_df)

,id,cell_id,cell_type,source,rank,ancestor_id
0,0009d135ece78d,ddfd239c,code,import numpy as np # linear algebra\nimport pa...,0,0
1,0009d135ece78d,c6cd22db,code,df = pd.read_csv('/kaggle/input/breast-cancer-...,1,0
2,0009d135ece78d,1372ae9b,code,"numerical_data = df.loc[:, ~df.columns.isin(['...",2,0
3,0009d135ece78d,90ed07ab,code,"def comparison_plot_maker(data_1, data_2, name...",3,0
4,0009d135ece78d,7f388a41,code,# Ploting data with different columns\n#######...,4,0
5,0009d135ece78d,2843a25a,code,# Scaling Data\nscaler = StandardScaler()\nsca...,5,0
6,0009d135ece78d,06dbf8cf,code,# 3. Implementing PCA on X (green for benign; ...,6,0
7,0009d135ece78d,f9893819,markdown,# Scaling Data ⚖\nLet's scale the data so PCA ...,0,0
8,0009d135ece78d,ba55e576,markdown,## Testing Plots >w>\nLet's these mystery soli...,1,0
9,0009d135ece78d,39e937ec,markdown,"## Plotting PCA 📊\nThus, the sun boils down to...",2,0


In [ ]:
tmp_df = original_df.groupby(["id","cell_type"])["cell_id"].agg("count").unstack().reset_index()
tmp_df["md_code_ratio"] = (tmp_df["markdown"] / (tmp_df["markdown"]+tmp_df["code"])).astype("float16")
tmp_df["code_ratio"] = (1/tmp_df["code"]).astype("float16")
tmp_df

cell_type,id,code,markdown,md_code_ratio,code_ratio
0,0009d135ece78d,7,6,0.461426,0.142822
1,0010483c12ba9b,9,1,0.099976,0.111084
2,0010a919d60e4f,27,35,0.564453,0.037048
3,0028856e09c5b7,3,1,0.250000,0.333252


In [ ]:
tmp1 = original_df[original_df["cell_type"]=="markdown"][["id", "cell_id", "source", "rank"]].rename(columns={"cell_id": "md_id", "source": "md_source", "rank": "md_rank"})
tmp2 = original_df[original_df["cell_type"]=="code"][["id", "cell_id", "source", "rank", "ancestor_id"]].rename(columns={"cell_id": "code_id", "source": "code_source", "rank": "code_rank"})
df = pd.merge(tmp1, tmp2, on="id", how="outer")
df = pd.merge(df, tmp_df[["id", "md_code_ratio", "code_ratio"]], on="id", how="left")

In [ ]:
#df["upvote"] = df["md_source"].progress_apply(lambda x: 1 if "upvote" in x.lower() else 0).astype("int8")
#df["fork"] = df["md_source"].progress_apply(lambda x: 1 if "fork" in x.lower() else 0).astype("int8")

In [ ]:
df

,id,md_id,md_source,md_rank,code_id,code_source,code_rank,ancestor_id,md_code_ratio,code_ratio
0,0009d135ece78d,f9893819,# Scaling Data ⚖\nLet's scale the data so PCA ...,0,ddfd239c,import numpy as np # linear algebra\nimport pa...,0,0,0.461426,0.142822
1,0009d135ece78d,f9893819,# Scaling Data ⚖\nLet's scale the data so PCA ...,0,c6cd22db,df = pd.read_csv('/kaggle/input/breast-cancer-...,1,0,0.461426,0.142822
2,0009d135ece78d,f9893819,# Scaling Data ⚖\nLet's scale the data so PCA ...,0,1372ae9b,"numerical_data = df.loc[:, ~df.columns.isin(['...",2,0,0.461426,0.142822
3,0009d135ece78d,f9893819,# Scaling Data ⚖\nLet's scale the data so PCA ...,0,90ed07ab,"def comparison_plot_maker(data_1, data_2, name...",3,0,0.461426,0.142822
4,0009d135ece78d,f9893819,# Scaling Data ⚖\nLet's scale the data so PCA ...,0,7f388a41,# Ploting data with different columns\n#######...,4,0,0.461426,0.142822
...,...,...,...,...,...,...,...,...,...,...
994,0010a919d60e4f,d3f5c397,We have 177 rows with missing `Age` and 687 ro...,34,177f908c,data_to_test = scaler.transform(test_data[feat...,25,0,0.564453,0.037048
995,0010a919d60e4f,d3f5c397,We have 177 rows with missing `Age` and 687 ro...,34,4356ab34,"\nestimators = [#('SVM',SVM_model.best_estimat...",26,0,0.564453,0.037048
996,0028856e09c5b7,eb293dfc,"automated to with data [Future you Sales code,...",0,012c9d02,"sns.set()\nsns.pairplot(data1, 2.5)\nplt.show(...",0,0,0.250000,0.333252
997,0028856e09c5b7,eb293dfc,"automated to with data [Future you Sales code,...",0,d22526d1,"types----------"")\n# is uniques----------"")\n...",1,0,0.250000,0.333252


In [ ]:
if CFG.train:
    df["label"] = df["code_rank"] - df["md_rank"]
    df["label"] = (df["label"]==1).astype("int8")
    df["code_pct"] = df.groupby(["id", "md_id"])["code_rank"].cumcount()
    df["code_pct"] = (df["code_pct"] / df.groupby(["id", "md_id"])["code_pct"].transform("max")).fillna(0.5)
else:
    df["label"] = 0
    df["code_pct"] = (df["code_rank"] / df.groupby(["id", "md_id"])["code_rank"].transform("max")).fillna(0.5)

In [ ]:
if not CFG.train:
    if len(original_df)<=89:
        CFG.data_split = 2
    kf = GroupKFold(n_splits=CFG.data_split)
    for i_fold, (trn_idx, val_idx) in enumerate(kf.split(df, df["id"], df["id"])):
        df.loc[val_idx, "data_split"] = i_fold
    df["data_split"] = df["data_split"].astype("int8")
    display(df)

,id,md_id,md_source,md_rank,code_id,code_source,code_rank,ancestor_id,md_code_ratio,code_ratio,label,code_pct,data_split
0,0009d135ece78d,f9893819,# Scaling Data ⚖\nLet's scale the data so PCA ...,0,ddfd239c,import numpy as np # linear algebra\nimport pa...,0,0,0.461426,0.142822,0,0.000000,1
1,0009d135ece78d,f9893819,# Scaling Data ⚖\nLet's scale the data so PCA ...,0,c6cd22db,df = pd.read_csv('/kaggle/input/breast-cancer-...,1,0,0.461426,0.142822,0,0.166667,1
2,0009d135ece78d,f9893819,# Scaling Data ⚖\nLet's scale the data so PCA ...,0,1372ae9b,"numerical_data = df.loc[:, ~df.columns.isin(['...",2,0,0.461426,0.142822,0,0.333333,1
3,0009d135ece78d,f9893819,# Scaling Data ⚖\nLet's scale the data so PCA ...,0,90ed07ab,"def comparison_plot_maker(data_1, data_2, name...",3,0,0.461426,0.142822,0,0.500000,1
4,0009d135ece78d,f9893819,# Scaling Data ⚖\nLet's scale the data so PCA ...,0,7f388a41,# Ploting data with different columns\n#######...,4,0,0.461426,0.142822,0,0.666667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0010a919d60e4f,d3f5c397,We have 177 rows with missing `Age` and 687 ro...,34,177f908c,data_to_test = scaler.transform(test_data[feat...,25,0,0.564453,0.037048,0,0.961538,0
995,0010a919d60e4f,d3f5c397,We have 177 rows with missing `Age` and 687 ro...,34,4356ab34,"\nestimators = [#('SVM',SVM_model.best_estimat...",26,0,0.564453,0.037048,0,1.000000,0
996,0028856e09c5b7,eb293dfc,"automated to with data [Future you Sales code,...",0,012c9d02,"sns.set()\nsns.pairplot(data1, 2.5)\nplt.show(...",0,0,0.250000,0.333252,0,0.000000,1
997,0028856e09c5b7,eb293dfc,"automated to with data [Future you Sales code,...",0,d22526d1,"types----------"")\n# is uniques----------"")\n...",1,0,0.250000,0.333252,0,0.500000,1


# Data Cleansing

In [ ]:
#original_df["source"] = original_df["source"].progress_apply(lambda x: re.sub("\n", "", x))
#display(original_df)

# CV Split

In [ ]:
if CFG.train:
    kf = GroupKFold(n_splits=CFG.fold)
    for i_fold, (trn_idx, val_idx) in enumerate(kf.split(df, df["ancestor_id"], df["ancestor_id"])):
        df.loc[val_idx, "fold"] = i_fold
    df["fold"] = df["fold"].astype("int8")
    print(df["fold"].value_counts())

In [ ]:
def get_sub(valid_df):

    valid_df["pred"] = np.exp((valid_df["pred"] - valid_df.groupby(["id", "md_id"])["pred"].transform("mean"))*25)
    valid_df["pred"] = valid_df["pred"] / valid_df.groupby(["id", "md_id"])["pred"].transform("sum")
    valid_df["rank"] = valid_df["pred"] * (valid_df["code_rank"]-0.5)
    valid_df = valid_df.groupby(["id", "md_id"])["rank"].agg("sum").reset_index()

    sub_df = pd.concat([
        valid_df[["id", "md_id", "rank"]].rename(columns={"md_id": "cell_id"}),
        original_df[original_df["id"].isin(valid_df["id"].unique())].query("cell_type=='code'")[["id", "cell_id", "rank"]]
    ]).reset_index(drop=True)

    sub_df = sub_df.sort_values("rank", ascending=True).groupby("id")["cell_id"].apply(lambda x: " ".join(x)).reset_index()
    sub_df.rename(columns={"cell_id": "cell_order"}, inplace=True)

    return sub_df

# Dataset

In [ ]:
# ====================================================
#  Dataset
# ====================================================

class AI4CodeDataset(Dataset):

    def __init__(self, df, include_labels=True):
        super().__init__()

        self.df = df.reset_index(drop=True)
        self.include_labels = include_labels

        self.md_source = df["md_source"].tolist()
        self.code_source = df["code_source"].tolist()
        self.code_rank = df["code_pct"].tolist()
        self.md_code_ratio = df["md_code_ratio"].tolist()
        self.code_ratio = df["code_ratio"].tolist()

        if self.include_labels:
            self.label = df["label"].tolist()

        self.tokenizer = AutoTokenizer.from_pretrained(CFG.model)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        text = self.md_source[idx] + '[SEP]' + self.code_source[idx]

        encoded = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens = True,
            max_length = CFG.max_length,
            padding = "max_length",
            return_token_type_ids = True,
            truncation = True
        )

        input_ids = torch.tensor(encoded['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(encoded['attention_mask'], dtype=torch.long)

        code_rank = torch.tensor(self.code_rank[idx], dtype=torch.float16)
        md_code_ratio = torch.tensor(self.md_code_ratio[idx], dtype=torch.float16)
        code_ratio = torch.tensor(self.code_ratio[idx], dtype=torch.float16)

        if self.include_labels:
            label = torch.tensor(self.label[idx], dtype=torch.float16)
            return input_ids, attention_mask, code_rank, md_code_ratio, code_ratio, label

        return input_ids, attention_mask, code_rank, md_code_ratio, code_ratio

# Model

In [ ]:
# ====================================================
#  Model
# ====================================================

class CustomModel(nn.Module):
    def __init__(self, model_name):
        super(CustomModel, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.head = nn.Sequential(
            nn.Linear(self.config.hidden_size+3, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            #nn.Dropout(0.1), # 0.2
            nn.Linear(64, 1),
        )

    def forward(self, ids, mask, code_rank, md_ratio, cos_sim):
        code_rank, md_ratio, cos_sim = code_rank.view(-1, 1), md_ratio.view(-1, 1), cos_sim.view(-1, 1)
        out = self.model(ids, mask)
        out = out[0][:, 0, :] # CLS Token
        out = torch.cat([out, code_rank, md_ratio, cos_sim], axis=1)
        out = self.head(out)
        return out

print(CustomModel(CFG.model))

CustomModel(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)

In [ ]:
def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler):
    
    start = end = time.time()
    losses = AverageMeter()
    scores = AverageMeter()

    model.train()

    scaler = torch.cuda.amp.GradScaler()

    for step, (input_ids, attention_mask, code_rank, md_code_ratio, code_ratio, label) in enumerate(train_loader):

        input_ids = input_ids.to(device, dtype=torch.long)
        attention_mask = attention_mask.to(device, dtype=torch.long)
        code_rank = code_rank.to(device, dtype=torch.float16)
        md_code_ratio = md_code_ratio.to(device, dtype=torch.float16)
        code_ratio = code_ratio.to(device, dtype=torch.float16)
        label = label.to(device, dtype=torch.float16)

        batch_size = label.size(0)

        with torch.cuda.amp.autocast():
            pred = model(input_ids, attention_mask, code_rank, md_code_ratio, code_ratio)
            loss = criterion(pred.view(-1), label)

        # record loss
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()

        scaler.step(optimizer)
        scaler.update()

        #optimizer.step()
        optimizer.zero_grad()

        # step
        scheduler.step()
                
        if CFG.scheduler=='ReduceLROnPlateau':
            lr = optimizer.param_groups[0]['lr']
        else:
            lr = scheduler.get_lr()[0]

        if step % 1000 == 0 or step == (len(train_loader) - 1):
            LOGGER.info(
                f"Epoch: [{epoch + 1}][{step}/{len(train_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(train_loader)):s} "
                f"Loss: {losses.avg:.6f} "
                f"LR: {lr:.8f} "
            )

    return losses.avg

def valid_fn(valid_loader, model, criterion, epoch):

    model.eval()
    
    start = end = time.time()
    losses = AverageMeter()

    preds = []
    
    for step, (input_ids, attention_mask, code_rank, md_code_ratio, code_ratio, label) in enumerate(valid_loader):

        input_ids = input_ids.to(device, dtype=torch.long)
        attention_mask = attention_mask.to(device, dtype=torch.long)
        code_rank = code_rank.to(device, dtype=torch.float16)
        md_code_ratio = md_code_ratio.to(device, dtype=torch.float16)
        code_ratio = code_ratio.to(device, dtype=torch.float16)
        label = label.to(device, dtype=torch.float16)

        batch_size = label.size(0)

        with torch.no_grad():
            pred = model(input_ids, attention_mask, code_rank, md_code_ratio, code_ratio)

        loss = criterion(pred.view(-1), label)
        losses.update(loss.item(), batch_size)

        preds.append(pred.view(-1).sigmoid().detach().cpu().numpy())

        end = time.time()
        if step % 1000 == 0 or step == (len(valid_loader)-1):
            LOGGER.info(
                f"EVAL: [{epoch + 1}][{step}/{len(valid_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(valid_loader)):s} "
                f"Loss: {losses.avg:.6f} "
            )

    preds = np.concatenate(preds)

    return losses.avg, preds

def inference_fn(test_loader, model):

    model.eval()
    model.to(device)
    
    preds = []

    for step, (input_ids, attention_mask, code_rank, md_code_ratio, code_ratio, _) in tqdm(enumerate(test_loader), total=len(test_loader)):
        
        input_ids = input_ids.to(device, dtype=torch.long)
        attention_mask = attention_mask.to(device, dtype=torch.long)
        code_rank = code_rank.to(device, dtype=torch.float16)
        md_code_ratio = md_code_ratio.to(device, dtype=torch.float16)
        code_ratio = code_ratio.to(device, dtype=torch.float16)
        
        with torch.no_grad():
            pred = torch.sigmoid(model(input_ids, attention_mask, code_rank, md_code_ratio, code_ratio))
        preds.append(pred.view(-1).detach().cpu().numpy())
    preds = np.concatenate(preds)
    
    return preds

In [ ]:
# ====================================================
# helper function
# ====================================================

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

In [ ]:
def train(df):

    for fold in range(CFG.fold):
        if fold in CFG.used_fold:

            LOGGER.info(f"==============================================")
            LOGGER.info(f"▶︎ Start Training fold{fold}")
            LOGGER.info(f"==============================================")

            train = df.query(f"fold!={fold}").reset_index(drop=True)

            # Negative Sampling
            train = pd.concat([
                            train[train["label"]==1],
                            train[train["label"]==0].sample(frac=0.05, random_state=CFG.seed)
                        ]).reset_index(drop=True)

            valid = df.query(f"fold=={fold}").reset_index(drop=True)

            # Sample valid data to save time
            valid = pd.concat([
                            valid[valid["label"]==1],
                            valid[valid["label"]==0].sample(frac=0.05, random_state=CFG.seed)
                        ]).reset_index(drop=True)

            # ====================================================
            #  Data Loader
            # ====================================================

            train_dataset = AI4CodeDataset(train)
            valid_dataset = AI4CodeDataset(valid)

            train_loader = DataLoader(
                train_dataset,
                batch_size=CFG.batch_size,
                shuffle=True,
                num_workers=CFG.num_workers,
                pin_memory=True,
                drop_last=False,
                #worker_init_fn=seed_worker,
                #generator=g
            )

            valid_loader = DataLoader(
                valid_dataset,
                batch_size=CFG.batch_size * 8,
                shuffle=False,
                num_workers=CFG.num_workers,
                pin_memory=True,
                drop_last=False,
                #worker_init_fn=seed_worker,
                #generator=g
            )

            # ====================================================
            #  Model
            # ====================================================
            model = CustomModel(CFG.model)
            model.to(device)

            param_optimizer = list(model.named_parameters())

            optimizer = AdamW(model.parameters(), lr=CFG.lr)
            num_train_steps = len(train_loader)* CFG.epochs

            def get_scheduler(optimizer):
                if CFG.scheduler=='linear':
                    scheduler = get_linear_schedule_with_warmup(
                        optimizer, num_warmup_steps=num_train_steps*0.05, num_training_steps=num_train_steps
                    )
                elif CFG.scheduler=='cosine':
                    scheduler = get_cosine_schedule_with_warmup(
                        optimizer, num_warmup_steps=CFG.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=CFG.num_cycles
                    )
                elif CFG.scheduler=='ReduceLROnPlateau':
                    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
                elif CFG.scheduler=='CosineAnnealingLR':
                    scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
                elif CFG.scheduler=='CosineAnnealingWarmRestarts':
                    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=CFG.T_mult, eta_min=CFG.min_lr, last_epoch=-1)
                elif CFG.scheduler=='CosineAnnealingWarmupRestarts':
                    scheduler = CosineAnnealingWarmupRestarts(optimizer, first_cycle_steps=CFG.first_cycle_steps, cycle_mult=CFG.cycle_mult, max_lr=CFG.lr, min_lr=CFG.min_lr, gamma=CFG.gamma, last_epoch=-1)
                return scheduler

            scheduler = get_scheduler(optimizer)

            criterion = nn.BCEWithLogitsLoss() # [nn.BCEWithLogitsLoss(), nn.L1Loss(), nn.MSELoss()]

            # ====================================================
            #  Loop
            # ====================================================

            best_loss = np.inf

            for epoch in range(CFG.epochs):
                start_time = time.time()

                # train
                avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler)

                # eval
                avg_val_loss, preds = valid_fn(valid_loader, model, criterion, epoch)

                # scoring

                elapsed = time.time() - start_time
                LOGGER.info(
                    f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.6f}  avg_valid_loss: {avg_val_loss:.6f} time: {elapsed:.0f}s"
                )

                # ====================================================
                #  Judge if the best score or not
                # ====================================================
                
                if avg_val_loss < best_loss:
                    best_loss = avg_val_loss
                    LOGGER.info(f"Epoch {epoch+1} - Best Loss")
                    torch.save({'model': model.state_dict(),
                                    'preds': preds},
                                    OUTPUT_DIR+f"fold{fold}_best.pth")

            del model, train, valid
            torch.cuda.empty_cache()
            gc_clear()
    
    # ==============================================
    # Create Kaggle Dataset
    # ==============================================

    !kaggle datasets init -p $OUTPUT_DIR

    metadata = {"id": f"shkanda/ai4code-dataset-exp{CFG.exp}",
                    "title": f"ai4code-dataset-exp{CFG.exp}",
                    "licenses": [{"name": "CC0-1.0"}]}

    with open(OUTPUT_DIR+'dataset-metadata.json', 'w') as fp:
        json.dump(metadata, fp)

    !kaggle datasets create -p $OUTPUT_DIR

# Train

In [ ]:
if CFG.train:
    train(df)

# Inference

In [ ]:
def predict(df):
    
    new_df = pd.DataFrame()
    
    for i in range(CFG.data_split):
        tmp_df = df[df["data_split"]==i].reset_index(drop=True)

        test_dataset = AI4CodeDataset(tmp_df)

        test_loader = DataLoader(test_dataset, 
                                 batch_size=512,
                                 shuffle=False, 
                                 num_workers=CFG.num_workers,
                                 pin_memory=True,
                                 drop_last=False,
                                 #worker_init_fn=seed_worker,
                                 #generator=g
                                 )
        preds = []

        for fold in range(CFG.fold):
            if fold in CFG.used_fold:
                
                model = CustomModel(CFG.model)
                model_path = MODEL_DIR + f"fold{fold}_best.pth"
                state = torch.load(model_path, map_location=torch.device('cpu'))
                model.load_state_dict(state["model"])
                
                pred = inference_fn(test_loader, model)
                
                preds.append(pred)
                
                del model
                gc_clear()
                
        tmp_df["pred"] = np.mean(preds, axis=0)
        tmp_df = get_sub(tmp_df)
            
        new_df = new_df.append(tmp_df, ignore_index=True)
        
        del tmp_df
        gc_clear()
                                
    return new_df

In [ ]:
if not CFG.train:
    sub_df = predict(df)
    sub_df.to_csv("submission.csv", index=False)
    display(sub_df)

100%|██████████| 1/1 [00:00<00:00,  1.46it/s]


,id,cell_order
0,0010a919d60e4f,b7578789 23607d04 aafc3d23 bbff12d4 80e077ec 8...
1,0009d135ece78d,0a226b6a ddfd239c 8cb8d28a c6cd22db 1372ae9b e...
2,0010483c12ba9b,7f270e34 54c7cab3 fe66203e 7844d5f8 5ce8863c 4...
3,0028856e09c5b7,012c9d02 eb293dfc d22526d1 3ae7ece3
